In [1]:
import time
import json

In [13]:
f = open('data/demo1.json')

In [14]:
data = json.load(f)

In [21]:
f2 = open('data/demo2.json')

In [22]:
data2 = json.load(f2)


In [5]:
my_data = list()

In [6]:
for frame in data:
    # 转换成localtime
    a = int(frame['timeStamp']/1000)
    # a = int(frame['timeStamp'])
    time_local = time.localtime(a)
    # 转换成新的时间格式(2016-05-05 20:28:54)
    dt = time.strftime("%Y-%m-%d %H:%M:%S.", time_local)
    frame['timeStamp'] = dt + str(frame['timeStamp'])[10:13]
    my_data.append(frame)

In [11]:
# my_data

In [12]:
with open('data/demo2.json', 'w') as f:
    json.dump(my_data, f, indent=2)
f.close()


In [26]:
f2.close()

In [25]:
f.close()

In [27]:
f = open('data/0427-501.txt')

In [28]:
lines = f.readlines()

In [31]:
total = list()
single_user = list()
id_count = 1

In [52]:
for line in lines:
    i = 0
    line = line.replace("\"", "'")
    T_count = line.count("[")
    print("T_count: ", T_count, "id_count:", id_count)
    while i < T_count:
        # 从标识符'['与']'之间提取出数据
        a = line.split("[")[i+1]
        a = a.split("]")[0]
        if a != "":
            if a.count('name') == 1 | a.count('scrollDirectionY') == 1:
                single_user.append(eval(a))
            else:
                single_user.extend(eval(a))
        i += 1

    if len(single_user):  # 排除空行
        # 给每一行的user赋一个ID（编号从1开始）
        for frame in single_user:
            frame['user'] = id_count
        id_count += 1
        # 按照时间顺序为所有字典排序
        single_user.sort(key=lambda x: x["timeStamp"])
        # 将每个用户的数据加入到total里
        total.extend(single_user)
    else:
        print("空行")
    single_user.clear()


print("total: ", total)
# 保存原始数据
with open('data/test.json', 'w') as f:
    json.dump(total, f, indent=2)

f.close()

T_count:  3 id_count: 1
total:  [{'name': 'listScroll', 'timeStamp': 1619497537774, 'args': "{'scrollDirectionX':'unchanged','scrollDirectionY':'forward','scrollDuration':679,'scrollEndOffsetX':0,'scrollEndOffsetY':933,'scrollEndTime':1619497538453,'scrollSpeedX':0,'scrollSpeedY':1.374079528718704,'scrollStartOffsetX':0,'scrollStartOffsetY':0,'scrollStartTime':1619497537774}", 'pSession': '1619497410598', 'scene': 'Page_SearchItemList', 'session': '158520404Page_SearchItemList', 'user': 1}, {'name': 'listScroll', 'timeStamp': 1619497543582, 'args': "{'scrollDirectionX':'unchanged','scrollDirectionY':'forward','scrollDuration':2257,'scrollEndOffsetX':0,'scrollEndOffsetY':1531,'scrollEndTime':1619497545839,'scrollSpeedX':0,'scrollSpeedY':0.26495347806823216,'scrollStartOffsetX':0,'scrollStartOffsetY':933,'scrollStartTime':1619497543582}", 'pSession': '1619497410598', 'scene': 'Page_SearchItemList', 'session': '158520404Page_SearchItemList', 'user': 1}, {'name': 'listScroll', 'timeStamp':

In [58]:
f = open('data/test.json')
# 将json文件作为python字典读入
data = json.load(f)

my_data = list()

In [60]:
# for frame in data:
#     del frame['pSession']
#     del frame['session']
#     if 'spm' in frame:
#         del frame['spm']
#     if 'spm-cnt' in frame:
#         del frame['spm-cnt']

In [ ]:
for frame in data:
    # del frame['pSession']
    # del frame['session']
    # if 'spm' in frame:
    #     del frame['spm']
    # if 'spm-cnt' in frame:
    #     del frame['spm-cnt']
    if 'name' in frame:
        # 判断是否为scroll
        if 'scrollDirectionY' in (frame['args']):
            # frame['act'] = 'scrolls'    # 滑动帧
            arg_dic = eval(frame['args'])
            # del frame['args']
            # # 左滑退出
            # if arg_dic['scrollDirectionX'] == 'negative':
            #     frame['type'] = 'Exit'
            #     frame['state'] = 3
            # 下滑浏览
            if arg_dic['scrollDirectionY'] == 'forward':
                frame['type'] = 'ScrollDown'
                frame['state'] = 2
                my_data.append(frame)
            # 上滑回看
            if arg_dic['scrollDirectionY'] == 'negative':
                frame['type'] = 'ScrollUp'
                frame['state'] = 1
                my_data.append(frame)
            # 这种情况还没考虑
            if arg_dic['scrollDirectionY'] == 'unchanged' and arg_dic['scrollDirectionX'] == 'unchanged':
                print("scroll:unchanged")
                frame['type'] = 'ScrollUnchanged'
            # 根据需要可以提取arg的scrollDuration以及scrollSpeed
        else:
            # del frame['args']
            frame['type'] = 'tap'
            frame['state'] = 0  # 此处默认点击代表进入下一级页面
            my_data.append(frame)
    else:
        # page_view帧
        del frame['fromScene']
        del frame['toScene']
        # 不太确定这fromScene和toScene的信息要不要删除
        my_data.append(frame)


In [65]:
f = open('data/demo.json')
# 将json文件作为python字典读入
data = json.load(f)
my_data = list()
id_num = 0
cur_scene = ''

In [69]:
for frame in data:
    # 判断是否进入下一个ID
    if frame['user'] != id_num:
        cur_scene = ''
        id_num = frame['user']

    if 'type' in frame:  # tap or scroll
        frame['scene'] = cur_scene
        my_data.append(frame)   # 只剩下tap和scroll
    else:                # pv
        cur_scene = frame['scene']


In [71]:
# for frame in data:
#     print(frame,'\n')

In [72]:
f = open('data/demo1.json')
# 将json文件作为python字典读入
data = json.load(f)

In [73]:
# data

[{'type': 'exit',
  'scene': 'Page_MYTBSettingVC',
  'state': 3,
  'user': 1,
  'timeStamp': 1616543038150},
 {'type': 'exit',
  'scene': 'Page_MyTaobao',
  'state': 3,
  'user': 1,
  'timeStamp': 1616543038291},
 {'name': '',
  'user': 1,
  'state': 0,
  'timeStamp': 1616543039850,
  'type': 'tap',
  'scene': 'https:\\/\\/market.m.taobao.com\\/app\\/tmall-def\\/juhuasuan-d2c\\/web\\/index.html'},
 {'name': '',
  'user': 1,
  'state': 0,
  'timeStamp': 1616543042483,
  'type': 'tap',
  'scene': 'https:\\/\\/market.m.taobao.com\\/app\\/tmall-def\\/juhuasuan-d2c\\/web\\/index.html'},
 {'name': '',
  'user': 1,
  'state': 0,
  'timeStamp': 1616543043190,
  'type': 'tap',
  'scene': 'https:\\/\\/pages.tmall.com\\/wow\\/a\\/act\\/ju\\/dailygroup\\/1527\\/wupr'},
 {'name': '',
  'user': 1,
  'state': 0,
  'timeStamp': 1616543043560,
  'type': 'tap',
  'scene': 'https:\\/\\/market.m.taobao.com\\/app\\/tmall-def\\/juhuasuan-d2c\\/web\\/index.html'},
 {'name': '',
  'user': 1,
  'state': 0,
  '